__Data preprocessing of the raw train-test data, to remove any duplicates, label conflicts, or 
IC50 value errors__

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import sys
sys.path.append("../../src/common")
from pharmacy_common import PharmacyCommon
common = PharmacyCommon()

# Starting data preprocessing

## 1. Read the data

In [2]:
all_data_path = "../../data/raw_data/raw_data_XO.xlsx"
dataset = pd.read_excel(all_data_path, sheet_name='All_dataset')

In [3]:
dataset_c = dataset.copy()
print(len(dataset))
dataset.head()

483


,CID,SMILES,IC50(nM),aid,Positive control substance,IC50_positive (nM),Type,Substructure,Natural products,Plant species,Unnamed: 10,Code,Substructure.1
0,190,C1=NC2=NC=NC(=C2N1)N,10890.0,287937,Allopurinol,7820.0,inactive,3,plant,water extract of wheat leaf,NaN,1,"1,2,4-Triazole"
1,471,C1=CC(=C(C=C1C2C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,100000.0,399340,Allopurinol,240.0,inactive,14,flavonoids,NaN,NaN,2,catechol
2,675,CC1=CC2=C(C=C1C)N=CN2,200000.0,287937,Allopurinol,7820.0,inactive,16,plant,water extract of wheat leaf,NaN,3,pyrimidine
3,938,C1=CC(=CN=C1)C(=O)O,518230.0,1444598,Allopurinol,2000.0,inactive,16,NaN,NaN,NaN,4,1H-pyrazole-4-carbonitrile
4,4947,CCCOC(=O)C1=CC(=C(C(=C1)O)O)O,628000.0,378145,Allopurinol,7450.0,inactive,2,plant,Boswellia papyrifera,NaN,5,4-methylthiazole-5-carboxylic acid


In [4]:
ic50_col_name = "IC50(nM)"
activity_col_name = "Type"
smiles_col_name = "SMILES"
cid_col_name = "CID"

In [5]:
def show_activity_distribution(label, dataset):
    #Rows for specific labels
    active_rows = dataset.loc[dataset[label] == "active"]
    inactive_rows = dataset.loc[dataset[label] == "inactive"]
    inconclusive_rows = dataset.loc[dataset[label] == "inconclusive"]
    unspecified_rows = dataset.loc[dataset[label] == "unspecified"]
    dataset_length = len(dataset)
    print("Total dataset")
    table = [['', 'active', 'inactive', 'inconclusive', 'unspecified'], 
            ['Number', len(active_rows), len(inactive_rows), len(inconclusive_rows), len(unspecified_rows)],
            ['Percentage (%)', len(active_rows)/dataset_length*100, len(inactive_rows)/dataset_length*100,
            len(inconclusive_rows)/dataset_length*100, len(unspecified_rows)/dataset_length*100]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

## 2. Activity distribution

We used to have inconclusive and unspecified labels in the raw data, however, those labels in the raw data have been removed by now.

In [6]:
show_activity_distribution(dataset=dataset_c, label=activity_col_name)

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   active │   inactive │   inconclusive │   unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 217      │   266      │              0 │             0 │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │  44.9275 │    55.0725 │              0 │             0 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


## 3. Remove conflict labels data

In [7]:
def check_label_intersection(dataset, col_name):
    active_rows = dataset.loc[dataset[col_name] == "active"]
    inactive_rows = dataset.loc[dataset[col_name] == "inactive"]
    inconclusive_rows = dataset.loc[dataset[col_name] == "inconclusive"]
    unspecified_rows = dataset.loc[dataset[col_name] == "unspecified"]
    
    cid_active = active_rows.loc[:, 'SMILES']
    cid_inactive = inactive_rows.loc[:, 'SMILES']
    cid_incon = inconclusive_rows.loc[:, 'SMILES']
    cid_unspec = unspecified_rows.loc[:, 'SMILES']
    
    ac_inac_cid = np.intersect1d(cid_active, cid_inactive)
    ac_incon_cid = np.intersect1d(cid_active, cid_incon)
    ac_unspec_cid = np.intersect1d(cid_active, cid_unspec)

    inac_incon_cid = np.intersect1d(cid_inactive, cid_incon)
    incon_unspec_cid = np.intersect1d(cid_incon, cid_unspec)
    inac_unspec_cid = np.intersect1d(cid_inactive, cid_unspec)
    
    print("Activity intersection:")
    table = [['Active-Inactive', 'Active-Inconclusive', 'Active-Unspecified', 'Inactive-Inconclusive', 'Inactive-Unspecified', 'Inconclusive-Unspecifid'], 
             [len(ac_inac_cid), len(ac_incon_cid), len(ac_unspec_cid), len(inac_incon_cid), len(inac_unspec_cid), len(incon_unspec_cid)]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [8]:
check_label_intersection(dataset=dataset_c, col_name=activity_col_name)

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


# Remove error data

In [9]:
def find_non_float_ic50(dataset, ic50_col_name):
    # Use to_numeric to identify non-float values
    is_float = pd.to_numeric(dataset[ic50_col_name], errors='coerce').notna()
    # Find the rows where 'IC50 (nM)' is not a float
    non_float_rows = dataset[~is_float]
    return non_float_rows

dataset[ic50_col_name] = pd.to_numeric(dataset[ic50_col_name])
dataset = dataset.dropna(subset=[ic50_col_name])
dataset = dataset.drop(dataset.loc[dataset[ic50_col_name] == 0].index)

In [10]:
show_activity_distribution(dataset=dataset_c, label=activity_col_name)

Total dataset
╒════════════════╤══════════╤════════════╤════════════════╤═══════════════╕
│                │   active │   inactive │   inconclusive │   unspecified │
╞════════════════╪══════════╪════════════╪════════════════╪═══════════════╡
│ Number         │ 217      │   266      │              0 │             0 │
├────────────────┼──────────┼────────────┼────────────────┼───────────────┤
│ Percentage (%) │  44.9275 │    55.0725 │              0 │             0 │
╘════════════════╧══════════╧════════════╧════════════════╧═══════════════╛


In [11]:
non_float_rows = find_non_float_ic50(dataset=dataset_c, ic50_col_name=ic50_col_name)
non_float_rows.head()

,CID,SMILES,IC50(nM),aid,Positive control substance,IC50_positive (nM),Type,Substructure,Natural products,Plant species,Unnamed: 10,Code,Substructure.1


In [12]:
print(len(dataset_c.loc[dataset_c[ic50_col_name] == 0].index))

0


## Remove labels intersection

__Remove label intersection__

In [13]:
def check_label_intersection(dataset, col_name):
    '''
        Description: Search for any data that have the same SMILES but have different labels
    '''
    active_rows = dataset.loc[dataset[col_name] == "active"]
    inactive_rows = dataset.loc[dataset[col_name] == "inactive"]
    inconclusive_rows = dataset.loc[dataset[col_name] == "inconclusive"]
    unspecified_rows = dataset.loc[dataset[col_name] == "unspecified"]
    
    cid_active = active_rows.loc[:, 'SMILES']
    cid_inactive = inactive_rows.loc[:, 'SMILES']
    cid_incon = inconclusive_rows.loc[:, 'SMILES']
    cid_unspec = unspecified_rows.loc[:, 'SMILES']
    
    ac_inac_cid = np.intersect1d(cid_active, cid_inactive)
    ac_incon_cid = np.intersect1d(cid_active, cid_incon)
    ac_unspec_cid = np.intersect1d(cid_active, cid_unspec)

    inac_incon_cid = np.intersect1d(cid_inactive, cid_incon)
    incon_unspec_cid = np.intersect1d(cid_incon, cid_unspec)
    inac_unspec_cid = np.intersect1d(cid_inactive, cid_unspec)
    
    print("Activity intersection:")
    table = [['Active-Inactive', 'Active-Inconclusive', 'Active-Unspecified', 'Inactive-Inconclusive', 'Inactive-Unspecified', 'Inconclusive-Unspecifid'], 
             [len(ac_inac_cid), len(ac_incon_cid), len(ac_unspec_cid), len(inac_incon_cid), len(inac_unspec_cid), len(incon_unspec_cid)]]
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [14]:
check_label_intersection(dataset=dataset_c, col_name=activity_col_name)

Activity intersection:
╒═══════════════════╤═══════════════════════╤══════════════════════╤═════════════════════════╤════════════════════════╤═══════════════════════════╕
│   Active-Inactive │   Active-Inconclusive │   Active-Unspecified │   Inactive-Inconclusive │   Inactive-Unspecified │   Inconclusive-Unspecifid │
╞═══════════════════╪═══════════════════════╪══════════════════════╪═════════════════════════╪════════════════════════╪═══════════════════════════╡
│                 0 │                     0 │                    0 │                       0 │                      0 │                         0 │
╘═══════════════════╧═══════════════════════╧══════════════════════╧═════════════════════════╧════════════════════════╧═══════════════════════════╛


# Remove duplicate smiles

In [15]:
def remove_dup_smiles(dataset, smile_col_name):
    unique_result = pd.DataFrame(columns=dataset.columns)
    dup_result = pd.DataFrame(columns=dataset.columns)
    error_result = pd.DataFrame(columns=dataset.columns)
    #Start iteration
    unique_smiles = dataset[smile_col_name].unique()
    for smile in unique_smiles:
        sub_dataset = dataset.loc[dataset[smile_col_name] == smile]
        if(len(sub_dataset) == 1):
            unique_result = pd.concat([unique_result, sub_dataset], axis=0)
        elif(len(sub_dataset) > 1):
            dup_result = pd.concat([dup_result, sub_dataset], axis=0)
        else:
            error_result = pd.concat([error_result, sub_dataset], axis=0)
    return unique_result, dup_result, error_result

In [16]:
unique_smiles = dataset_c[smiles_col_name].unique()
len(unique_smiles)

483

In [17]:
unique_result, dup_result, error_result = remove_dup_smiles(dataset=dataset_c, smile_col_name=smiles_col_name)

/tmp/ipykernel_305685/2184178721.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unique_result = pd.concat([unique_result, sub_dataset], axis=0)


In [18]:
print(len(unique_result), len(dup_result), len(error_result))

483 0 0


In [19]:
new_data = unique_result
print(len(new_data))
new_data.head()

483


,CID,SMILES,IC50(nM),aid,Positive control substance,IC50_positive (nM),Type,Substructure,Natural products,Plant species,Unnamed: 10,Code,Substructure.1
0,190,C1=NC2=NC=NC(=C2N1)N,10890.0,287937,Allopurinol,7820.0,inactive,3,plant,water extract of wheat leaf,NaN,1,"1,2,4-Triazole"
1,471,C1=CC(=C(C=C1C2C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,100000.0,399340,Allopurinol,240.0,inactive,14,flavonoids,NaN,NaN,2,catechol
2,675,CC1=CC2=C(C=C1C)N=CN2,200000.0,287937,Allopurinol,7820.0,inactive,16,plant,water extract of wheat leaf,NaN,3,pyrimidine
3,938,C1=CC(=CN=C1)C(=O)O,518230.0,1444598,Allopurinol,2000.0,inactive,16,NaN,NaN,NaN,4,1H-pyrazole-4-carbonitrile
4,4947,CCCOC(=O)C1=CC(=C(C(=C1)O)O)O,628000.0,378145,Allopurinol,7450.0,inactive,2,plant,Boswellia papyrifera,NaN,5,4-methylthiazole-5-carboxylic acid


__Check if the SMILES are encodable__

In [20]:
maccs_fpts = common.gen_maccs_fpts(new_data['SMILES'])
ecfp4_1024bits_fpts = common.gen_ecfp4_fpts(new_data['SMILES'], 1024)
ecfp4_2048bits_fpts = common.gen_ecfp4_fpts(new_data['SMILES'], 2048)
ecfp6_1024bits_fpts = common.gen_ecfp6_fpts(new_data['SMILES'], 1024)
ecfp6_2048bits_fpts = common.gen_ecfp6_fpts(new_data['SMILES'], 2048)

Progress:   0%|          | 0/483 [00:00<?, ?it/s][15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:12] DEPRECATION WARNING: please use MorganGenerator
[15:17:

In [30]:
new_data.head()

,CID,SMILES,IC50(nM),aid,Positive control substance,IC50_positive (nM),Type,Substructure,Natural products,Plant species,Unnamed: 10,Code,Substructure.1
0,190,C1=NC2=NC=NC(=C2N1)N,10890.0,287937,Allopurinol,7820.0,inactive,3,plant,water extract of wheat leaf,NaN,1,"1,2,4-Triazole"
1,471,C1=CC(=C(C=C1C2C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,100000.0,399340,Allopurinol,240.0,inactive,14,flavonoids,NaN,NaN,2,catechol
2,675,CC1=CC2=C(C=C1C)N=CN2,200000.0,287937,Allopurinol,7820.0,inactive,16,plant,water extract of wheat leaf,NaN,3,pyrimidine
3,938,C1=CC(=CN=C1)C(=O)O,518230.0,1444598,Allopurinol,2000.0,inactive,16,NaN,NaN,NaN,4,1H-pyrazole-4-carbonitrile
4,4947,CCCOC(=O)C1=CC(=C(C(=C1)O)O)O,628000.0,378145,Allopurinol,7450.0,inactive,2,plant,Boswellia papyrifera,NaN,5,4-methylthiazole-5-carboxylic acid


In [21]:
final_data = new_data[['CID', 'SMILES', 'IC50(nM)', 'aid', 'Type', 'Substructure']]
final_data.head(5)

,CID,SMILES,IC50(nM),aid,Type,Substructure
0,190,C1=NC2=NC=NC(=C2N1)N,10890.0,287937,inactive,3
1,471,C1=CC(=C(C=C1C2C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,100000.0,399340,inactive,14
2,675,CC1=CC2=C(C=C1C)N=CN2,200000.0,287937,inactive,16
3,938,C1=CC(=CN=C1)C(=O)O,518230.0,1444598,inactive,16
4,4947,CCCOC(=O)C1=CC(=C(C(=C1)O)O)O,628000.0,378145,inactive,2


__Write to file__

In [ ]:
with pd.ExcelWriter('../../data/preprocessed_data/preprocessed_dataset_XO.xlsx', engine='openpyxl') as writer:
    dataset_c.to_excel(writer, sheet_name='All_dataset', index=False)
    final_data.to_excel(writer, sheet_name='Preprocessed_dataset', index=False)